<a href="https://colab.research.google.com/github/azhara001/cs182hw8/blob/main/Copy_of_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://raw.githubusercontent.com/Berkeley-CS182/cs182fa23_public/main/q_wandbai/architectures.py
!pip install wandb

--2023-10-31 01:37:42--  https://raw.githubusercontent.com/Berkeley-CS182/cs182fa23_public/main/q_wandbai/architectures.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1618 (1.6K) [text/plain]
Saving to: ‘architectures.py’

architectures.py    100%[===================>]   1.58K  --.-KB/s    in 0s      

2023-10-31 01:37:42 (23.5 MB/s) - ‘architectures.py’ saved [1618/1618]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for pat

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from architectures import BasicConvNet, ResNet18, MLP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Exploring Tensorboard
Tensorboard is a local tool for visualizing images, metrics, histograms, and more. It is designed for tensorflow, but can be integrated with torch. Let's explore tensorboard usage with an example:

```
from torch.utils.tensorboard import SummaryWriter

# To start a run, call the following
writer = SummaryWriter(comment=f'Name_of_Run')

# When you want to log a value, use the writer. When adding a scalar, the format is as follows:
# add_scalar(tag, scalar_value, global_step=None, walltime=None, new_style=False, double_precision=False)
writer.add_scalar('Training Loss', loss.item(), step)

# Finally, when you are done logging values, close the writer
writer.close()
```
There are many other functionalities and methods that you are free to explore, but will not be mentioned in this notebook.

## Your Task
We will be once again building classifiers for the CIFAR-10. There are various architectures set up for you to use in the architectures.py file. Using tensorboard, please search through 5 different hyperparameter configurations. Examples of choices include: learning rate, batch size, architecture, optimization algorithm, etc. Please submit the generated plots on your pdf and answer question A.

In [6]:
epochs = 2
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
print(device)

cpu


In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


100%|██████████| 170498071/170498071 [00:03<00:00, 45570788.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [24]:
hyperparameters = {
    'learning_rate' : [1e-2],#, 1e-2, 1e-1, 1, 1e1],
    'optimizer:' : ['SGD', 'ADAM','AdamW']
}

class BasicConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [46]:
from torch.utils.data.dataloader import DataLoader

def run(lr=1e-3, batch_size=128):
    # Perhaps you want to make a function to train on a certain set of hyperparameters
    # Don't forget to use tensorboard
    writer = SummaryWriter(comment=f'lr:{lr}_batch_size:{batch_size}')
    model = BasicConvNet().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=lr)

    train_loader = DataLoader(trainset,batch_size=batch_size,shuffle=True)
    test_loader = DataLoader(testset,batch_size=batch_size,shuffle=True)
    step=0
    for _ in range(int(epochs*128/batch_size)):
      dataiter = iter(test_loader)
      for i, data in enumerate(train_loader):
        X,Y = data
        X,Y = X.to(device),Y.to(device)
        optimizer.zero_grad()

        output = model(X)
        loss = loss_fn(output,Y)
        loss.backward()
        optimizer.step()
        accuracy = torch.mean((torch.argmax(output, dim=1) == Y).float()).item() * 100

        if i % log_freq == 0:
              writer.add_scalar('Training Loss', loss.item(), step)
              writer.add_scalar('Training Accuracy', accuracy, step)
        if i % eval_freq == 0:
            eval_inputs, eval_labels = next(dataiter)
            eval_inputs, eval_labels = eval_inputs.to(device), eval_labels.to(device)
            #eval_inputs = torch.flatten(eval_inputs, start_dim=1)
            model.eval()
            with torch.no_grad():
                eval_outputs = model(eval_inputs)
            eval_loss = loss_fn(eval_outputs, eval_labels)
            model.train()
            eval_accuracy = torch.mean((torch.argmax(eval_outputs, dim=1) == eval_labels).float()).item() * 100
            writer.add_scalar('Validation Loss', eval_loss.item(), step)
            writer.add_scalar('Validation Accuracy', eval_accuracy, step)
        step += 1
        print(f'\rStep: {step}', end='')
    print(f'\nRun with (lr: {lr}, batch_size: {batch_size}) has finished')
    writer.close()

In [47]:
learning_rates = [1e-3, 1e-4]
batch_sizes = [128, 256]
log_freq = 2
eval_freq = 5

for lr in learning_rates:
    for batch_size in batch_sizes:
        run(lr, batch_size)


Step: 782
Run with (lr: 0.001, batch_size: 128) has finished
Step: 196
Run with (lr: 0.001, batch_size: 256) has finished
Step: 782
Run with (lr: 0.0001, batch_size: 128) has finished
Step: 196
Run with (lr: 0.0001, batch_size: 256) has finished
